## Mutex: Mutual Exclusion for Shared State

Về mặt chữ viết thì `Metex` là viết tắt của `Mut`al `Ex`clusion, đại ý là chỉ một process hoặc thread truy cập shared resource tại một thời điểm mà thôi, để tránh `race condition` hoặc `data corruption`

Chúng ta có channel (giới thiệu ở phần 02) cho triết lý của Go: "Don't communicate by sharing memory; share memory by communicating", nhưng trong vài trường hợp nếu cần handle một shared resource nào đó, thì chúng ta vẫn phải dùng Mutex.

## When to Use Mutex vs Channels

**Use Channels when:**
- Passing ownership of data
- Distributing work to multiple goroutines
- Communicating async results

**Dùng Mutex khi:**
- Bảo vệ trạng thái bên trong của một struct 
- Cache lại những kết quả từ những phần nặng tính toán
- Quản lý shared resources

## Understanding Mutex

Mutext của Go đảm bảo rằng chỉ 1 goroutine có thể truy cập 1 resource nào đó tại một thời điểm. 

In [ ]:
var (
    counter int
    mu      sync.Mutex
)

func increment() {
    mu.Lock()
    counter++
    mu.Unlock()
}

## Basic Mutex Patterns

### 1. Simple Counter

In [1]:
type Counter struct {
    mu    sync.Mutex
    value int
}

func (c *Counter) Inc() {
    c.mu.Lock()
    defer c.mu.Unlock()
    c.value++
}

func (c *Counter) Value() int {
    c.mu.Lock()
    defer c.mu.Unlock()
    return c.value
}

func main() {
    var wg sync.WaitGroup
    counter := &Counter{}
    
    for i := 0; i < 1000; i++ {
        wg.Add(1)
        go func() {
            defer wg.Done()
            counter.Inc()
        }()
    }
    
    wg.Wait()
    fmt.Println("Final count:", counter.Value()) // Always 1000
}

Final count: 1000


Đầu tiên, chúng ta khai báo `value` và `mu` trong cùng 1 struct, điều này không băt buộc nhưng đây là best practice
```go
type Counter struct {
    mu    sync.Mutex
    value int
}
```

Chúng ta có 2 func là `Inc()` để tăng value và `Value()` để đọc value. Dù func Value() dùng để đọc nhưng cũng có mutext lock. Giả sử như Goroutine A tăng value, mà Goroutine B đọc value thì sẽ có data race.

Tiếp theo là Waitgroup được giới thiệu ở phần 09, mỗi vòng lặp sẽ có `wg.Add` có input là 1 (kiểu như tăng bộ đếm lên 1), khi go func thực hiện xong thì sẽ có `wg.Done()` để báo xong (kiểu như giảm bộ đếm đi 1). Và `wg.Wait` sẽ chờ cho tới khi bộ đếm về không, ngăn không do Goroutine main thoát sớm. Kết quả cuối cùng luôn là 1000, dù 1000 Goroutine này chạy song song để tăng value lên 1 với mỗi Goroutine.

### 2. Read-Write Mutex

In [ ]:
type Cache struct {
    mu    sync.RWMutex
    data  map[string]string
}

func (c *Cache) Get(key string) (string, bool) {
    c.mu.RLock()
    defer c.mu.RUnlock()
    val, ok := c.data[key]
    return val, ok
}

func (c *Cache) Set(key, value string) {
    c.mu.Lock()
    defer c.mu.Unlock()
    c.data[key] = value
}

// Multiple readers can proceed simultaneously
// Writers get exclusive access

#### sync.Mutex vs sync.RWMutex

Ở ví dụ phần 1, chúng ta thấy mutext trong Counter struct là `sync.Mutex` còn ở ví dụ này là `sync.RWMutex`.   
- `sync.Mutex`: giống như chìa khóa nhà vệ sinh, một người ở trong, những người khác phải chờ ở ngoài. Nó không quan tâm đến việc bạn chỉ nhìn hay sử dụng, nó chỉ quan tâm rằng có đúng 1 người ở bên trong.
- `sync.RWMutex` là một mutext được chuyên dùng cho việc đọc (Read), nó có 2 kiểu lock:
  - `RLock()`: nhiều Goroutine có thể đọc (Read) cùng một lúc
  - `Lock()`: chỉ có 1 Goroutine có thể ghi (Write)

- `Get` để đọc (Read) dữ liệu, nên sẽ sử dụng `RLock`, cho nhiều Goroutine có thể đọc dữ liệu cùng lúc
- `Set` để ghi (Write) dữ liệu, nên sẽ sự dụng `Lock`, chỉ duy nhất một Goroutine có thể ghi dữ liệu mà thôi

#### Ví dụ dưới để ví dụ 1 writer goroutine và 5 reader goroutines

In [2]:
package main

import (
	"fmt"
	"math/rand"
	"sync"
	"time"
)

type Cache struct {
	mu   sync.RWMutex
	data map[string]string
}

func NewCache() *Cache {
	return &Cache{
		data: make(map[string]string),
	}
}

func (c *Cache) Get(key string) (string, bool) {
	c.mu.RLock()
	defer c.mu.RUnlock()

	val, ok := c.data[key]
	return val, ok
}

func (c *Cache) Set(key, value string) {
	c.mu.Lock()
	defer c.mu.Unlock()

	c.data[key] = value
}

func main() {
	rand.Seed(time.Now().UnixNano())

	cache := NewCache()

	// Preload some data
	cache.Set("a", "apple")
	cache.Set("b", "banana")

	var wg sync.WaitGroup

	// 1 writer goroutine: updates values periodically
	wg.Add(1)
	go func() {
		defer wg.Done()

		keys := []string{"a", "b", "c"}
		values := []string{"ant", "boat", "cat", "dog", "egg"}

		for i := 0; i < 10; i++ {
			key := keys[rand.Intn(len(keys))]
			val := values[rand.Intn(len(values))]

			fmt.Printf("[WRITER] wants Lock()  key=%q val=%q\n", key, val)
			cache.mu.Lock()
			fmt.Printf("[WRITER] got   Lock()  key=%q val=%q (exclusive)\n", key, val)

			// Simulate slow write work (holding the lock)
			cache.data[key] = val
			time.Sleep(120 * time.Millisecond)

			cache.mu.Unlock()
			fmt.Printf("[WRITER] released Lock() key=%q\n", key)

			time.Sleep(80 * time.Millisecond)
		}
	}()

	// 5 reader goroutines: continuously read
	readerCount := 5
	for r := 1; r <= readerCount; r++ {
		wg.Add(1)
		go func(id int) {
			defer wg.Done()

			keys := []string{"a", "b", "c"}
			for i := 0; i < 20; i++ {
				key := keys[rand.Intn(len(keys))]

				fmt.Printf("  [R%d] wants RLock() key=%q\n", id, key)
				cache.mu.RLock()
				fmt.Printf("  [R%d] got   RLock() key=%q (shared)\n", id, key)

				// Simulate slow read work (holding the read lock)
				val, ok := cache.data[key]
				time.Sleep(60 * time.Millisecond)

				cache.mu.RUnlock()
				if ok {
					fmt.Printf("  [R%d] released RLock() key=%q -> %q\n", id, key, val)
				} else {
					fmt.Printf("  [R%d] released RLock() key=%q -> (missing)\n", id, key)
				}

				time.Sleep(40 * time.Millisecond)
			}
		}(r)
	}

	wg.Wait()
	fmt.Println("Done.")
}


  [R5] wants RLock() key="a"
  [R5] got   RLock() key="a" (shared)
  [R1] wants RLock() key="a"
  [R1] got   RLock() key="a" (shared)
  [R4] wants RLock() key="b"
  [R4] got   RLock() key="b" (shared)
  [R2] wants RLock() key="c"
  [R2] got   RLock() key="c" (shared)
[WRITER] wants Lock()  key="b" val="cat"
  [R3] wants RLock() key="c"
  [R4] released RLock() key="b" -> "banana"
  [R1] released RLock() key="a" -> "apple"
  [R5] released RLock() key="a" -> "apple"
  [R2] released RLock() key="c" -> (missing)
[WRITER] got   Lock()  key="b" val="cat" (exclusive)
  [R2] wants RLock() key="a"
  [R5] wants RLock() key="b"
  [R1] wants RLock() key="c"
  [R4] wants RLock() key="a"
[WRITER] released Lock() key="b"
  [R4] got   RLock() key="a" (shared)
  [R5] got   RLock() key="b" (shared)
  [R2] got   RLock() key="a" (shared)
  [R1] got   RLock() key="c" (shared)
  [R3] got   RLock() key="c" (shared)
  [R3] released RLock() key="c" -> (missing)
  [R1] released RLock() key="c" -> (missing)
  [R5

### 3. Lazy Initialization

In [ ]:
type Singleton struct {
    mu   sync.Mutex
    data *expensiveResource
}

func (s *Singleton) Get() *expensiveResource {
    s.mu.Lock()
    defer s.mu.Unlock()
    
    if s.data == nil {
        s.data = createExpensiveResource()
    }
    return s.data
}

**Sơ lược đại khái về Singleton pattern**:
- Nó được xếp vào `creational` design pattern, là pattern đơn giản mà hầu như ai cũng nghe qua
- Mỗi class chỉ có 1 instance
- Instance này là static, được tạo 1 lần duy nhất lúc khởi tạo (first request) và nó được truy cập global

Ở ví dụ trên:
- Khi `Get()` được gọi, `createExpensiveResource` sẽ được gọi 1 lần duy nhất khi s.data là nil, và được gán vào s.data
- Method này sẽ không được gọi lần nào nữa, vì s.data được gán giá trị rồi nên sẽ không vào trong if khi check nil

In [ ]:
// Better: Use sync.Once
type BetterSingleton struct {
    once sync.Once
    data *expensiveResource
}

func (s *BetterSingleton) Get() *expensiveResource {
    s.once.Do(func() {
        s.data = createExpensiveResource()
    })
    return s.data
}

**sync.Once** - `once.Do(f)` đảm bảo rằng
- Hàm f bên trong chỉ chạy đúng 1 lần, cho dù nhiều Goroutine cùng chạy `Get()`
- Mọi Goroutine phải chờ khi có 1 Goroutine đang chạy
- Sau khi done thì các lần gọi `Do` sau này sẽ return ngay lập tức

## Advanced Mutex Patterns (Pending)

## Common Mutex Pitfalls